In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_National_Hockey_League_arenas"
g = requests.get(url)

In [3]:
soup = BeautifulSoup(g.text, "html5lib")
wiki_arena_list = soup.find("tbody")
wiki_column = wiki_arena_list.find("tr").text
wiki_column=wiki_column.split("\n")

In [4]:
for i in range(len(wiki_column)): 
    try: 
        print(i, wiki_column[i])
        if wiki_column[i] == "": 
            wiki_column.pop(i)
    except:
        pass

0 
1 
2 
3 
4 
5 
6 
7 
8 


In [5]:
wiki_column

['Image',
 'Arena',
 'Location',
 'Team',
 'Capacity',
 'Opened',
 'Season of first NHL game',
 'Ref(s)']

In [6]:
wiki_cells = wiki_arena_list.find_all("td")

In [7]:
col0=[]
col1=[]
col2=[]
col3=[]
col4=[]
col5=[]
col6=[]
col7=[]

for i in range(len(wiki_cells)): 
    if i%8 == 0: 
        col0.append(wiki_cells[i].text[0:-1])     
    elif i%8==1: 
        col1.append(wiki_cells[i].text[0:-1])
    elif i%8==2: 
        col2.append(wiki_cells[i].text[0:-1])
    elif i%8==3: 
        col3.append(wiki_cells[i].text[0:-1])
    elif i%8==4: 
        col4.append(wiki_cells[i].text[0:-1])
    elif i%8==5: 
        col5.append(wiki_cells[i].text[0:-1])
    elif i%8==6: 
        col6.append(wiki_cells[i].text[0:-1])
    else: 
        col7.append(wiki_cells[i].text[0:-1])

In [8]:
df=pd.DataFrame({wiki_column[0]:col0,
                 wiki_column[1]:col1,
                wiki_column[2]:col2,
                wiki_column[3]:col3,
                wiki_column[4]:col4,
                wiki_column[5]:col5,
                wiki_column[6]:col6,
                wiki_column[7]:col7,})

In [9]:
df.head()

,Image,Arena,Location,Team,Capacity,Opened,Season of first NHL game,Ref(s)
0,,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning,"19,092",1996,1996–97,[1]
1,,American Airlines Center,"Dallas, Texas",Dallas Stars,"18,532",2001,2001–02,[2]
2,,Ball Arena,"Denver, Colorado",Colorado Avalanche,"17,809",1999,1999–2000,[3]
3,,BB&T Center,"Sunrise, Florida",Florida Panthers,"19,250",1998,1998–99,[4]
4,,Bell Centre,"Montreal, Quebec",Montreal Canadiens,"21,302",1996,1995–96,[5]


In [10]:
df.drop("Ref(s)", axis = 1 ,inplace = True)

In [11]:
import geocoder
from geopy.geocoders import Nominatim

In [12]:
address = 'North America'
geolocator = Nominatim(user_agent="google")

In [13]:
location = geolocator.geocode('{}, {}'.format(df["Arena"][0],df["Location"][0]))
latitude = location.latitude
longitude = location.longitude

In [14]:
latitude=[]
longitude=[]

for k in range(32):
    try: 
        location = geolocator.geocode('{}, {}'.format(df["Arena"][k],df["Location"][k]))
        print(k,location, location.latitude,location.longitude)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except: 
        print(k)
        latitude.append(np.nan)
        longitude.append(np.nan)
        continue

0 Amalie Arena, 401, Channelside Drive, Harbour Island, Tampa, Hillsborough County, Florida, 33602, United States 27.942704 -82.45189031562487
1 American Airlines Center, 2500, Victory Avenue, Harwood District, Dallas, Dallas County, Texas, 75219, United States 32.7905076 -96.81027213460834
2 Ball Arena, 1000, Chopper Circle, Auraria, Denver, Denver County, Colorado, 80204, United States 39.748683799999995 -105.00754401780362
3 BB&T Center, 1, Northwest 137th Way, Sunrise, Broward County, Florida, 33323, United States 26.15837015 -80.3254288749938
4 Centre Bell, Rue Drummond, René-Lévesque, Ville-Marie, Montréal, Agglomération de Montréal, Montréal (06), Québec, H3B 4W8, Canada 45.4960916 -73.56927782355446
5 Bridgestone Arena, 5th Avenue South, Nashville-Davidson, Davidson County, Tennessee, 37203, United States 36.1589806 -86.77838189265074
6 Canada Life, 60, Osborne Street North, Osborne Village, Roslyn, Fort Rouge–East Fort Garry, Winnipeg, Winnipeg (city), Manitoba, R3C1V3, Canada

In [15]:
# No.28th area doesnt have the information 
# used https://www.openstreetmap.org/ to check, no results either 
df.iloc[28,:]

Image                                         
Arena                                UBS Arena
Location                      Elmont, New York
Team                        New York Islanders
Capacity                                17,113
Opened                                    2021
Season of first NHL game               2021–22
Name: 28, dtype: object

In [16]:
df["Latitude"]=latitude
df["Longitude"] = longitude

In [17]:
df.head()

,Image,Arena,Location,Team,Capacity,Opened,Season of first NHL game,Latitude,Longitude
0,,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning,"19,092",1996,1996–97,27.942704,-82.451890
1,,American Airlines Center,"Dallas, Texas",Dallas Stars,"18,532",2001,2001–02,32.790508,-96.810272
2,,Ball Arena,"Denver, Colorado",Colorado Avalanche,"17,809",1999,1999–2000,39.748684,-105.007544
3,,BB&T Center,"Sunrise, Florida",Florida Panthers,"19,250",1998,1998–99,26.158370,-80.325429
4,,Bell Centre,"Montreal, Quebec",Montreal Canadiens,"21,302",1996,1995–96,45.496092,-73.569278


In [18]:
# According to wikipedia:https://en.wikipedia.org/wiki/UBS_Arena. The arena geolocation is 40.712094°N 73.727157°W
# AKA: 40.712094  -73.727157
df.iloc[28,7]=40.712094
df.iloc[28,8] = -73.727157
df.iloc[28,:]

Image                                         
Arena                                UBS Arena
Location                      Elmont, New York
Team                        New York Islanders
Capacity                                17,113
Opened                                    2021
Season of first NHL game               2021–22
Latitude                               40.7121
Longitude                             -73.7272
Name: 28, dtype: object

In [19]:
df.drop("Image",axis=1,inplace=True)
df.head()

,Arena,Location,Team,Capacity,Opened,Season of first NHL game,Latitude,Longitude
0,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning,"19,092",1996,1996–97,27.942704,-82.451890
1,American Airlines Center,"Dallas, Texas",Dallas Stars,"18,532",2001,2001–02,32.790508,-96.810272
2,Ball Arena,"Denver, Colorado",Colorado Avalanche,"17,809",1999,1999–2000,39.748684,-105.007544
3,BB&T Center,"Sunrise, Florida",Florida Panthers,"19,250",1998,1998–99,26.158370,-80.325429
4,Bell Centre,"Montreal, Quebec",Montreal Canadiens,"21,302",1996,1995–96,45.496092,-73.569278


In [20]:
#export to json
result = df.to_json(r"NHL_arena_Lat_Log.json")

In [21]:
df.columns.values

array(['Arena', 'Location', 'Team', 'Capacity', 'Opened',
       'Season of first NHL game', 'Latitude', 'Longitude'], dtype=object)

## connect with Foursquare API and select top-picks

In [22]:
import matplotlib.cm as cm 
import matplotlib.colors as colors 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import folium
import seaborn as sns 
%matplotlib inline

In [23]:
address2 = 'United States of America'
geolocator2 = Nominatim(user_agent="google")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude

In [24]:
map_na = folium.Map(location=[latitude2,longitude2],zoom_start = 4,width = 1000)
for lat, lng, arena, city, team in zip(df["Latitude"], df["Longitude"],df["Arena"],df["Location"],df["Team"]):
    label = "Team: {}, \n Arena: {},  City: {}".format(team,arena,city)
    label = folium.Popup(label,parse_html=True, max_width=200)
    folium.CircleMarker(
    [lat,lng], 
    radius = 6 ,
    popup = label , 
    color = "red",
    fill=True, 
    fill_color = "red",
    fill_capacity = 0.7,
    parse_html=True).add_to(map_na)
    
map_na

In [25]:
CLIENT_ID = 'T3LYSVVFWCXCPBIHPLDIXEPPS2JNTX2LACLOQU0MXTR5VYU5' # your Foursquare ID
CLIENT_SECRET = 'PBUOZ1LKV1D14RQKXICJZT2FXNXDZ43Q3VD1L4VO0HRLX2N2' # your Foursquare Secret
CODE = "KJ04IZ1JI3IAXIQ3BEPQ2JX2CJRC4PY1DMEFNI2B0T0J1XK3#_=_"
ACCESS_TOKEN = 'IEZX0QMM1LAALBLWIDNJTGMZZN5SZ10WMU4T40YFC5FWZRVR' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T3LYSVVFWCXCPBIHPLDIXEPPS2JNTX2LACLOQU0MXTR5VYU5
CLIENT_SECRET:PBUOZ1LKV1D14RQKXICJZT2FXNXDZ43Q3VD1L4VO0HRLX2N2


In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
api_df = pd.DataFrame()

for c in range(32): 
    aa = df["Latitude"][c]
    bb = df["Longitude"][c]
    radius=4000
    section = "topPicks"

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&section={}'.format(CLIENT_ID, CLIENT_SECRET, aa, bb, VERSION, radius,section)
    results = requests.get(url).json()
    items = results["response"]["groups"][0]["items"]
    dataframe = pd.json_normalize(items) 

    filtered_columns = ["venue.name", "venue.categories"] + [col for col in dataframe.columns if col.startswith("venue.location.")]
    dataframe_filtered = dataframe.loc[:,filtered_columns]
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    dataframe_filtered["Arena"] = df["Arena"][c]
    dataframe_filtered["Location"]=df["Location"][c]
    dataframe_filtered["Team"] = df["Team"][c]
    
    api_df = pd.concat([api_df, dataframe_filtered])

In [28]:
api_df.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,Arena,Location,Team
0,Tampa Riverwalk,Scenic Lookout,Riverwalk,27.947131,-82.460898,"[{'label': 'display', 'lat': 27.94713088788469...",1013,33602,US,Tampa,FL,United States,"[Riverwalk, Tampa, FL 33602, United States]",NaN,NaN,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning
1,bartaco Hyde Park,Mexican Restaurant,1601 W Snow Ave,27.936253,-82.476194,"[{'label': 'display', 'lat': 27.9362528, 'lng'...",2495,33606,US,Tampa,FL,United States,"[1601 W Snow Ave, Tampa, FL 33606, United States]",NaN,NaN,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning
2,Buddy Brew Coffee,Coffee Shop,2020 W Kennedy Blvd,27.944645,-82.480715,"[{'label': 'display', 'lat': 27.94464481910003...",2842,33606,US,Tampa,FL,United States,"[2020 W Kennedy Blvd (at Albany Ave), Tampa, F...",at Albany Ave,NaN,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning
3,Hyde Park Village,Shopping Plaza,1602 W Snow Ave,27.936441,-82.476268,"[{'label': 'display', 'lat': 27.9364406, 'lng'...",2496,33606,US,Tampa,FL,United States,[1602 W Snow Ave (btwn S Swann & S Dakota Ave)...,btwn S Swann & S Dakota Ave,NaN,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning
4,Ulele,New American Restaurant,1836 N. Highland Ave.,27.959821,-82.462889,"[{'label': 'display', 'lat': 27.95982142071007...",2191,33602,US,Tampa,FL,United States,"[1836 N. Highland Ave. (Henderson St.), Tampa,...",Henderson St.,NaN,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning


In [29]:
nearby_map = folium.Map(location=[latitude2, longitude2], zoom_start = 4,width = 1000)

for lat, lng, arena, city, team in zip(df["Latitude"], df["Longitude"],df["Arena"],df["Location"],df["Team"]):
    label = "Team: {},  Arena: {},  City: {}".format(team,arena,city)
    label = folium.Popup(label,parse_html=True, max_width=200)
    folium.CircleMarker(
        [lat,lng], 
        radius = 20 ,
        popup = label , 
        color = "red",
        fill=True, 
        fill_color = "red",
        fill_capacity = 0.7).add_to(nearby_map)
    
for lat, lng, lb,storename in zip(api_df["lat"], api_df["lng"], api_df["categories"],api_df["name"]):
    label="name: {},  category: {}".format(storename, lb)
    label=folium.Popup(label,parse_html=True,max_width=200)
    folium.CircleMarker(
         [lat, lng],
         radius=5,
        popup=label,
         color='blue',
        fill=True,
         fill_color='blue',
         fill_opacity=0.6).add_to(nearby_map)

nearby_map

## cluster the arena based on the top-picks 

In [30]:
api_df.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'postalCode', 'cc', 'city', 'state', 'country',
       'formattedAddress', 'crossStreet', 'neighborhood', 'Arena', 'Location',
       'Team'],
      dtype='object')

In [31]:
api_df.shape

(480, 18)

In [32]:
len(api_df["categories"].unique())

129

In [33]:
cluster_onehot=pd.get_dummies(api_df[["categories"]],prefix="",prefix_sep="")
cluster_onehot.head()

,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Baseball Field,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
cluster_onehot.shape

(480, 129)

In [35]:
cluster_onehot["Arena"]=api_df["Arena"]
fixed_columns = [cluster_onehot.columns[-1]]+list(cluster_onehot.columns[:-1])

In [36]:
cluster_onehot = cluster_onehot[fixed_columns]
cluster_onehot.head()

,Arena,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop
0,Amalie Arena,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Amalie Arena,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Amalie Arena,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Amalie Arena,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amalie Arena,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
cluster = cluster_onehot.groupby("Arena").sum().reset_index()
cluster.shape

(32, 130)

In [38]:
cluster.head()

,Arena,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop
0,Amalie Arena,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,American Airlines Center,1,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,BB&T Center,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ball Arena,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Bell Centre,0,0,0,0,0,0,0,1,0,...,0,2,0,0,0,0,0,0,0,0


In [39]:
num_top_venues = 5

for hood in cluster['Arena']:
    print("----"+hood+"----")
    temp = cluster[cluster['Arena'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amalie Arena----
                 venue  freq
0  American Restaurant   2.0
1          Pizza Place   2.0
2       Scenic Lookout   2.0
3          Coffee Shop   2.0
4          Beer Garden   1.0


----American Airlines Center----
                 venue  freq
0              Brewery   4.0
1          Coffee Shop   2.0
2  American Restaurant   1.0
3         Gourmet Shop   1.0
4          Pizza Place   1.0


----BB&T Center----
                 venue  freq
0  American Restaurant   3.0
1                 Park   3.0
2       Clothing Store   2.0
3        Shopping Mall   2.0
4           Steakhouse   1.0


----Ball Arena----
                venue  freq
0             Brewery   3.0
1         Coffee Shop   2.0
2  Mexican Restaurant   1.0
3  Italian Restaurant   1.0
4          Restaurant   1.0


----Bell Centre----
                       venue  freq
0                   Tea Room   2.0
1                       Café   2.0
2  Middle Eastern Restaurant   1.0
3             Ice Cream Shop   1.0
4      Perform

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

columns = ['Arena']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

arena_venues_sorted = pd.DataFrame(columns=columns)
arena_venues_sorted['Arena'] = cluster['Arena']

for ind in np.arange(cluster.shape[0]):
    arena_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cluster.iloc[ind, :], num_top_venues)

arena_venues_sorted.head()

,Arena,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Amalie Arena,American Restaurant,Pizza Place,Coffee Shop,Scenic Lookout,Beer Garden,New American Restaurant,Movie Theater,Mexican Restaurant,Shopping Plaza,...,Brewery,Fried Chicken Joint,Frozen Yogurt Shop,Grocery Store,Cupcake Shop,Dance Studio,Deli / Bodega,Greek Restaurant,Dessert Shop,Diner
1,American Airlines Center,Brewery,Coffee Shop,American Restaurant,Gourmet Shop,Fast Food Restaurant,Seafood Restaurant,Bar,Shopping Mall,Art Museum,...,Pizza Place,Deli / Bodega,Farmers Market,Diner,Donut Shop,Dry Cleaner,Escape Room,Exhibit,Falafel Restaurant,Dance Studio
2,BB&T Center,American Restaurant,Park,Shopping Mall,Clothing Store,Italian Restaurant,Cuban Restaurant,Steakhouse,Caribbean Restaurant,Burger Joint,...,Dry Cleaner,Diner,Dessert Shop,Escape Room,Exhibit,Falafel Restaurant,Deli / Bodega,Dance Studio,Donut Shop,Fish & Chips Shop
3,Ball Arena,Brewery,Coffee Shop,Bar,Outdoor Sculpture,Cocktail Bar,Restaurant,Mexican Restaurant,Liquor Store,Italian Restaurant,...,Hot Dog Joint,Art Museum,German Restaurant,Falafel Restaurant,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Grocery Store,Greek Restaurant
4,Bell Centre,Café,Tea Room,Ice Cream Shop,Market,Lounge,Performing Arts Venue,Middle Eastern Restaurant,Jewelry Store,Music Venue,...,Bakery,Hotel,Plaza,Falafel Restaurant,Donut Shop,Dessert Shop,Dry Cleaner,Escape Room,Deli / Bodega,Dance Studio


In [42]:
kclusters = 4

a = cluster.drop('Arena', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(a)
kmeans.labels_[0:10] 

array([1, 2, 0, 2, 3, 1, 2, 1, 1, 2])

In [43]:
arena_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [44]:
arena_venues_sorted.shape

(32, 22)

In [45]:
df.head()

,Arena,Location,Team,Capacity,Opened,Season of first NHL game,Latitude,Longitude
0,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning,"19,092",1996,1996–97,27.942704,-82.451890
1,American Airlines Center,"Dallas, Texas",Dallas Stars,"18,532",2001,2001–02,32.790508,-96.810272
2,Ball Arena,"Denver, Colorado",Colorado Avalanche,"17,809",1999,1999–2000,39.748684,-105.007544
3,BB&T Center,"Sunrise, Florida",Florida Panthers,"19,250",1998,1998–99,26.158370,-80.325429
4,Bell Centre,"Montreal, Quebec",Montreal Canadiens,"21,302",1996,1995–96,45.496092,-73.569278


In [46]:
df_merged = df
df_merged = pd.merge(left = df_merged, right = arena_venues_sorted, on='Arena',how = "left")
df_merged.head()

,Arena,Location,Team,Capacity,Opened,Season of first NHL game,Latitude,Longitude,Cluster Labels,1st Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Amalie Arena,"Tampa, Florida",Tampa Bay Lightning,"19,092",1996,1996–97,27.942704,-82.451890,1,American Restaurant,...,Brewery,Fried Chicken Joint,Frozen Yogurt Shop,Grocery Store,Cupcake Shop,Dance Studio,Deli / Bodega,Greek Restaurant,Dessert Shop,Diner
1,American Airlines Center,"Dallas, Texas",Dallas Stars,"18,532",2001,2001–02,32.790508,-96.810272,2,Brewery,...,Pizza Place,Deli / Bodega,Farmers Market,Diner,Donut Shop,Dry Cleaner,Escape Room,Exhibit,Falafel Restaurant,Dance Studio
2,Ball Arena,"Denver, Colorado",Colorado Avalanche,"17,809",1999,1999–2000,39.748684,-105.007544,2,Brewery,...,Hot Dog Joint,Art Museum,German Restaurant,Falafel Restaurant,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Grocery Store,Greek Restaurant
3,BB&T Center,"Sunrise, Florida",Florida Panthers,"19,250",1998,1998–99,26.158370,-80.325429,0,American Restaurant,...,Dry Cleaner,Diner,Dessert Shop,Escape Room,Exhibit,Falafel Restaurant,Deli / Bodega,Dance Studio,Donut Shop,Fish & Chips Shop
4,Bell Centre,"Montreal, Quebec",Montreal Canadiens,"21,302",1996,1995–96,45.496092,-73.569278,3,Café,...,Bakery,Hotel,Plaza,Falafel Restaurant,Donut Shop,Dessert Shop,Dry Cleaner,Escape Room,Deli / Bodega,Dance Studio


In [47]:
df_merged.shape

(32, 29)

In [48]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']

In [49]:
cluster_map = folium.Map(location=[latitude2, longitude2], zoom_start = 4,width = 1000)

markers_colors = []
for lat, lon, arena, team, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Arena'],df_merged["Team"], df_merged['Cluster Labels']):
    label = folium.Popup(str(arena) +", Team: "+team+ ', Cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

In [50]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(8, df_merged.shape[1]))]]

,Location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
3,"Sunrise, Florida",0,American Restaurant,Park,Shopping Mall,Clothing Store,Italian Restaurant,Cuban Restaurant,Steakhouse,Caribbean Restaurant,...,Dry Cleaner,Diner,Dessert Shop,Escape Room,Exhibit,Falafel Restaurant,Deli / Bodega,Dance Studio,Donut Shop,Fish & Chips Shop
11,"Glendale, Arizona",0,American Restaurant,Pizza Place,Baseball Field,Clothing Store,Chinese Restaurant,Movie Theater,Brewery,Sporting Goods Shop,...,Whisky Bar,Golf Driving Range,German Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Gym,Donut Shop
27,"Paradise, Nevada",0,American Restaurant,Botanical Garden,Hotel,Hotel Pool,Japanese Restaurant,French Restaurant,Music Venue,Concert Hall,...,Snack Place,Comedy Club,Gym / Fitness Center,Taco Place,Fried Chicken Joint,Diner,Donut Shop,Athletics & Sports,Dry Cleaner,Escape Room


In [51]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(8, df_merged.shape[1]))]]

,Location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Tampa, Florida",1,American Restaurant,Pizza Place,Coffee Shop,Scenic Lookout,Beer Garden,New American Restaurant,Movie Theater,Mexican Restaurant,...,Brewery,Fried Chicken Joint,Frozen Yogurt Shop,Grocery Store,Cupcake Shop,Dance Studio,Deli / Bodega,Greek Restaurant,Dessert Shop,Diner
5,"Nashville, Tennessee",1,Coffee Shop,American Restaurant,Brewery,Indie Movie Theater,Italian Restaurant,Liquor Store,Fried Chicken Joint,Cupcake Shop,...,Restaurant,Gym / Fitness Center,Bar,Beer Garden,Frozen Yogurt Shop,Dry Cleaner,Escape Room,Exhibit,Falafel Restaurant,Farmers Market
7,"Ottawa, Ontario",1,Skating Rink,Restaurant,Coffee Shop,Outlet Mall,Italian Restaurant,Bar,Gastropub,Clothing Store,...,Vietnamese Restaurant,Pizza Place,Seafood Restaurant,Dance Studio,Exhibit,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Dry Cleaner
8,"Washington, D.C.",1,Art Museum,Coffee Shop,Cocktail Bar,Farmers Market,Wine Bar,Sculpture Garden,New American Restaurant,Indian Restaurant,...,French Restaurant,Garden,Restaurant,Donut Shop,Wine Shop,Diner,Escape Room,Dessert Shop,Deli / Bodega,Dance Studio
12,"Anaheim, California",1,Sandwich Place,American Restaurant,Snack Place,Hot Dog Joint,Lounge,Deli / Bodega,Gym / Fitness Center,Plaza,...,Brewery,Coffee Shop,Art Gallery,Theater,Dance Studio,Dessert Shop,Diner,Donut Shop,Dry Cleaner,Escape Room
16,"Columbus, Ohio",1,Bar,Coffee Shop,Market,Pizza Place,Restaurant,Brewery,German Restaurant,Park,...,Grocery Store,Art Gallery,Golf Driving Range,Exhibit,Cupcake Shop,Dance Studio,Deli / Bodega,Greek Restaurant,Dessert Shop,Diner
17,"Raleigh, North Carolina",1,Wine Shop,Italian Restaurant,Concert Hall,Pizza Place,Cocktail Bar,Chinese Restaurant,Sandwich Place,Sculpture Garden,...,Beer Store,Brewery,Sushi Restaurant,Arepa Restaurant,Art Museum,Garden,Deli / Bodega,Dessert Shop,Grocery Store,Diner
18,"Pittsburgh, Pennsylvania",1,Coffee Shop,Ice Cream Shop,Italian Restaurant,Korean Restaurant,State / Provincial Park,Sculpture Garden,Plaza,Brewery,...,Caribbean Restaurant,Argentinian Restaurant,History Museum,Pub,Escape Room,Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Diner
19,"Newark, New Jersey",1,Spanish Restaurant,Coffee Shop,Park,Tapas Restaurant,Lounge,Brazilian Restaurant,Sporting Goods Shop,Irish Pub,...,Deli / Bodega,Ice Cream Shop,Performing Arts Venue,Falafel Restaurant,Dessert Shop,Dance Studio,Cupcake Shop,Diner,Donut Shop,Dry Cleaner
21,"Edmonton, Alberta",1,Mexican Restaurant,Coffee Shop,Fast Food Restaurant,Café,Escape Room,Sporting Goods Shop,Nightclub,Bakery,...,Trail,Concert Hall,Grocery Store,Bar,Dessert Shop,Falafel Restaurant,Deli / Bodega,Diner,Donut Shop,Dry Cleaner


In [52]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(8, df_merged.shape[1]))]]

,Location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,"Dallas, Texas",2,Brewery,Coffee Shop,American Restaurant,Gourmet Shop,Fast Food Restaurant,Seafood Restaurant,Bar,Shopping Mall,...,Pizza Place,Deli / Bodega,Farmers Market,Diner,Donut Shop,Dry Cleaner,Escape Room,Exhibit,Falafel Restaurant,Dance Studio
2,"Denver, Colorado",2,Brewery,Coffee Shop,Bar,Outdoor Sculpture,Cocktail Bar,Restaurant,Mexican Restaurant,Liquor Store,...,Hot Dog Joint,Art Museum,German Restaurant,Falafel Restaurant,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Grocery Store,Greek Restaurant
6,"Winnipeg, Manitoba",2,Coffee Shop,Café,Brewery,Hotel Bar,Bookstore,Market,Fish & Chips Shop,Diner,...,Concert Hall,Bakery,Beer Bar,Arepa Restaurant,Art Gallery,Donut Shop,Dry Cleaner,Escape Room,Exhibit,Falafel Restaurant
9,"Seattle, Washington",2,Brewery,Playground,Taco Place,Market,Park,Cheese Shop,Museum,Frozen Yogurt Shop,...,Donut Shop,Bakery,Ice Cream Shop,Art Museum,Falafel Restaurant,Dry Cleaner,Diner,Escape Room,Dessert Shop,Deli / Bodega
10,"St. Louis, Missouri",2,Brewery,Coffee Shop,BBQ Joint,Bar,Park,Pub,Museum,Liquor Store,...,Gift Shop,Exhibit,Grocery Store,Dance Studio,Deli / Bodega,Dessert Shop,Greek Restaurant,Gourmet Shop,Diner,Donut Shop
13,"Buffalo, New York",2,Brewery,Bar,American Restaurant,Steakhouse,Pizza Place,Plaza,Pub,New American Restaurant,...,Burger Joint,Performing Arts Venue,Waterfront,Frozen Yogurt Shop,Fried Chicken Joint,Cuban Restaurant,Greek Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega
24,"Calgary, Alberta",2,Brewery,Restaurant,Exhibit,Coffee Shop,Sporting Goods Shop,Bakery,BBQ Joint,Park,...,Fast Food Restaurant,Diner,Donut Shop,Deli / Bodega,Dance Studio,Dry Cleaner,Escape Room,Cupcake Shop,Falafel Restaurant,Farmers Market
26,"Boston, Massachusetts",2,Brewery,Gastropub,Concert Hall,Coffee Shop,New American Restaurant,Seafood Restaurant,Mexican Restaurant,Park,...,Art Museum,Wine Bar,Pet Store,Bakery,Garden,Frozen Yogurt Shop,Dance Studio,Deli / Bodega,Dessert Shop,Grocery Store
31,"Saint Paul, Minnesota",2,Brewery,Pizza Place,Lounge,Organic Grocery,Museum,Science Museum,Candy Store,Burrito Place,...,American Restaurant,Italian Restaurant,Athletics & Sports,Garden,Golf Driving Range,Grocery Store,Dessert Shop,Greek Restaurant,Diner,Donut Shop


In [53]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(8, df_merged.shape[1]))]]

,Location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,"Montreal, Quebec",3,Café,Tea Room,Ice Cream Shop,Market,Lounge,Performing Arts Venue,Middle Eastern Restaurant,Jewelry Store,...,Bakery,Hotel,Plaza,Falafel Restaurant,Donut Shop,Dessert Shop,Dry Cleaner,Escape Room,Deli / Bodega,Dance Studio
14,"Detroit, Michigan",3,Cocktail Bar,American Restaurant,Beer Bar,Concert Hall,New American Restaurant,Café,Deli / Bodega,Gift Shop,...,Bakery,Gym,Art Gallery,Bar,Golf Driving Range,Farmers Market,Dessert Shop,Diner,Donut Shop,Gym / Fitness Center
15,"New York City, New York",3,Park,Art Gallery,Bookstore,Taco Place,Farmers Market,Thai Restaurant,Performing Arts Venue,Bakery,...,Escape Room,Falafel Restaurant,Exhibit,Cuban Restaurant,Cupcake Shop,Dry Cleaner,Diner,Fast Food Restaurant,Dessert Shop,Deli / Bodega
20,"Vancouver, British Columbia",3,Ice Cream Shop,Hotel,Gym / Fitness Center,Bakery,Coffee Shop,Ramen Restaurant,Market,Brewery,...,Park,Waterfront,Farmers Market,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Grocery Store,Donut Shop,Dry Cleaner
28,"Elmont, New York",3,Bakery,Pizza Place,Italian Restaurant,Liquor Store,Dry Cleaner,Caribbean Restaurant,Martial Arts School,Beer Garden,...,Greek Restaurant,Trail,Gastropub,Falafel Restaurant,Grocery Store,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner
